<a href="https://colab.research.google.com/github/solidjoe/E-Commerce_Sentiment_Intelligence/blob/main/E_Commerce_Sentiment_Intelligence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

!pip install pandas numpy scikit-learn nltk spacy tensorflow
!pip install requests


import pandas as pd
import numpy as np
import gzip
import json
from datetime import datetime
import requests
import ssl

In [3]:
# We will use the 'Electronics' category reviews as a manageable sample.
REVIEWS_URL = 'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz'


METADATA_URL = 'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Electronics.json.gz'


print("Downloading Reviews file (reviews_Electronics_5.json.gz)...")
!wget $REVIEWS_URL -O reviews.json.gz

print("Downloading Metadata file (meta_Electronics.json.gz)...")
!wget $METADATA_URL -O metadata.json.gz

print("\nDownload complete. Files are now in the Colab temporary storage.")

--2025-11-28 12:32:28--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 495854086 (473M) [application/x-gzip]
Saving to: ‘reviews.json.gz’

reviews.json.gz     100%[===================>] 472.88M  13.4MB/s    in 45s     

2025-11-28 12:33:13 (10.6 MB/s) - ‘reviews.json.gz’ saved [495854086/495854086]

--2025-11-28 12:33:14--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Electronics.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 186594679 (178M) [application/x-gzip]
Saving to: ‘metadata.json.gz’

metadata.json.gz    100%[===================>] 177.95M  12.0MB/s

In [4]:
# --- Helper Functions to Load Loose JSON Data ---

def parse(path):
    # Opens the gzipped file for reading in binary mode ('r').
    g = gzip.open(path, 'r')
    for l in g:
        # The data uses single quotes, which is not strict JSON, so we use eval().
        yield eval(l)

def get_dataframe(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    # Creates a DataFrame from the collected dictionaries.
    return pd.DataFrame.from_dict(df, orient='index')

# --- Load the Reviews Data ---
# Loading the Reviews file. This may take a minute or two.
print("Loading Reviews DataFrame...")
review_df = get_dataframe('reviews.json.gz')

print(f"Reviews Loaded: {review_df.shape[0]} total reviews.")
print("--- Sample Review Data ---")
print(review_df[['reviewText', 'overall', 'unixReviewTime']].head())

Loading Reviews DataFrame...
Reviews Loaded: 1689188 total reviews.
--- Sample Review Data ---
                                          reviewText  overall  unixReviewTime
0  We got this GPS for my husband who is an (OTR)...      5.0      1370131200
1  I'm a professional OTR truck driver, and I bou...      1.0      1290643200
2  Well, what can I say.  I've had this unit in m...      3.0      1283990400
3  Not going to write a long review, even thought...      2.0      1290556800
4  I've had mine for a year and here's what we go...      1.0      1317254400


In [5]:
# Convert the unix timestamp to a readable datetime object
review_df['reviewDate'] = pd.to_datetime(review_df['unixReviewTime'], unit='s')

# Define the starting date for the project (January 1, 2010)
START_DATE = datetime(2010, 1, 1)

# Filter the DataFrame to include only reviews from 2010 onwards
filtered_df = review_df[review_df['reviewDate'] >= START_DATE].copy()

# Add the 'year' column, essential for trend prediction later
filtered_df['year'] = filtered_df['reviewDate'].dt.year

print(f"\nReviews Filtered (2010-2014): {filtered_df.shape[0]} reviews.")
print(f"Time Range: {filtered_df['reviewDate'].min().date()} to {filtered_df['reviewDate'].max().date()}")


Reviews Filtered (2010-2014): 1494070 reviews.
Time Range: 2010-01-01 to 2014-07-23


In [6]:
# --- Load the Metadata Data ---
print("Loading Metadata DataFrame...")
metadata_df = get_dataframe('metadata.json.gz')

# --- Select and Clean Metadata Columns ---
metadata_df = metadata_df[['asin', 'categories']].copy()


def extract_main_category(categories):
    if categories and categories[0]:
        return categories[0][0]
    return 'Unknown'

metadata_df['main_category'] = metadata_df['categories'].apply(extract_main_category)
metadata_df = metadata_df.drop(columns=['categories']) # Drop the messy original column

# --- Merge the DataFrames ---
final_df = pd.merge(
    filtered_df,
    metadata_df,
    on='asin',
    how='left' # Keep all the filtered reviews, adding metadata where available
)

# --- Final Check of the Assembled Data ---
final_df = final_df[['reviewText', 'overall', 'year', 'main_category']].rename(
    columns={'overall': 'rating'}
)

final_df.dropna(subset=['reviewText', 'main_category', 'rating'], inplace=True)

print("\n--- Step 1: Data Collection COMPLETE ---")
print(f"Final Clean Dataset Shape: {final_df.shape}")
print(final_df.head())

Loading Metadata DataFrame...

--- Step 1: Data Collection COMPLETE ---
Final Clean Dataset Shape: (1494070, 4)
                                          reviewText  rating  year  \
0  We got this GPS for my husband who is an (OTR)...     5.0  2013   
1  I'm a professional OTR truck driver, and I bou...     1.0  2010   
2  Well, what can I say.  I've had this unit in m...     3.0  2010   
3  Not going to write a long review, even thought...     2.0  2010   
4  I've had mine for a year and here's what we go...     1.0  2011   

  main_category  
0   Electronics  
1   Electronics  
2   Electronics  
3   Electronics  
4   Electronics  


In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re


nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('punkt_tab', quiet=True) # THIS IS THE FIX for the LookupError

# Initialize the Lemmatizer and Stopwords list
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# --- Define the Cleaning Function ---
def clean_text(text):
    # 1. Lowercase
    text = text.lower()

    # 2. Remove Special Characters and Punctuation (Keep only letters and spaces)

    text = re.sub(r'[^a-z\s]', '', text)

    # 3. Tokenization: Split the text into individual words
    tokens = nltk.word_tokenize(text)

    # 4. Stopword Removal
    tokens = [word for word in tokens if word not in stop_words]

    # 5. Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # 6. Re-join tokens into a single string
    cleaned_text = ' '.join(tokens)

    return cleaned_text

# --- Apply the Cleaning Function to the DataFrame ---
print("Starting text cleaning... This may take several minutes for a large dataset.")
# Create a new column 'cleaned_review' to store the processed text


final_df['cleaned_review'] = final_df['reviewText'].apply(clean_text)

print("\nCleaning complete.")
print("Original Review Example:")
print(final_df['reviewText'].iloc[0])
print("\nCleaned Review Example:")
print(final_df['cleaned_review'].iloc[0])

Starting text cleaning... This may take several minutes for a large dataset.

Cleaning complete.
Original Review Example:
We got this GPS for my husband who is an (OTR) over the road trucker.  Very Impressed with the shipping time, it arrived a few days earlier than expected...  within a week of use however it started freezing up... could of just been a glitch in that unit.  Worked great when it worked!  Will work great for the normal person as well but does have the "trucker" option. (the big truck routes - tells you when a scale is coming up ect...)  Love the bigger screen, the ease of use, the ease of putting addresses into memory.  Nothing really bad to say about the unit with the exception of it freezing which is probably one in a million and that's just my luck.  I contacted the seller and within minutes of my email I received a email back with instructions for an exchange! VERY impressed all the way around!

Cleaned Review Example:
got gps husband otr road trucker impressed ship

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# --- 1. Initialize the TF-IDF Vectorizer ---

vectorizer = TfidfVectorizer(max_features=10000, min_df=5)

# --- 2. Fit and Transform the Cleaned Text ---
print("Starting TF-IDF Vectorization...")
# X will be our feature matrix
X = vectorizer.fit_transform(final_df['cleaned_review'])

# --- 3. Prepare Target Variables (Y) ---

y_rating = final_df['rating']
y_category = final_df['main_category']

# Display results
print("\nTF-IDF Vectorization complete.")
print(f"Shape of Feature Matrix (X): {X.shape}")

Starting TF-IDF Vectorization...

TF-IDF Vectorization complete.
Shape of Feature Matrix (X): (1494070, 10000)


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# --- 1. Define the Sentiment Target (Classification Task) ---

# We map the 1-5 star rating (y_rating) to these classes.
def map_rating_to_sentiment(rating):
    if rating >= 4:
        return 'Positive'
    elif rating == 3:
        return 'Neutral'
    else: # rating 1 or 2
        return 'Negative'

# Create the new sentiment target variable
y_sentiment = final_df['rating'].apply(map_rating_to_sentiment)

# --- 2. Split Data (for Classification and Predictive Modeling) ---
# We use X (the TF-IDF matrix) and y_sentiment (our new target).

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_sentiment,
    test_size=0.2,
    random_state=42,
    # Use the Stratify parameter to ensure training/test sets have the same proportion of classes
    stratify=y_sentiment
)

print(f"Total reviews: {X.shape[0]}")
print(f"Training set size: {X_train.shape[0]}")
print(f"Testing set size: {X_test.shape[0]}")
print(f"\nSentiment Distribution in Test Set:\n{y_test.value_counts(normalize=True)}")

Total reviews: 1494070
Training set size: 1195256
Testing set size: 298814

Sentiment Distribution in Test Set:
rating
Positive    0.806000
Negative    0.110149
Neutral     0.083851
Name: proportion, dtype: float64


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
import time

# --- List of Models to Evaluate (Optimized) ---
models = {
    # Logistic Regression already ran and was fast.
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    # Reduced n_estimators from 100 to 25 to significantly speed up training.
    'Random Forest Classifier (Optimized)': RandomForestClassifier(n_estimators=25, random_state=42, n_jobs=-1),
    # LinearSVC is still often slow, but necessary for the comparison.
    'Support Vector Machine (LinearSVC)': LinearSVC(random_state=42, dual=False, max_iter=1000)
}

print("Starting Classification Model Training and Evaluation (Optimized)...\n")
results = {}

for name, model in models.items():
    print(f"--- Training {name} ---")
    start_time = time.time()

    # Check if Logistic Regression result is available (from previous attempt)
    if name == 'Logistic Regression':
        # Skip retraining the completed model, use the output you already got
        accuracy = 0.8594
        report = """
              precision    recall  f1-score   support

    Negative     0.6922    0.5912    0.6377     32914
     Neutral     0.4553    0.1115    0.1791     25056
    Positive     0.8866    0.9739    0.9282    240844

    accuracy                         0.8594    298814
   macro avg     0.6780    0.5589    0.5817    298814
weighted avg     0.8290    0.8594    0.8334    298814
"""
    else:
        # Train the remaining models
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred, digits=4)

    end_time = time.time()

    # Store and print results
    results[name] = {'Accuracy': accuracy, 'Report': report}
    print(f"Training Time: {end_time - start_time:.2f} seconds")
    print(f"Test Accuracy for {name}: {accuracy:.4f}")
    print(f"Classification Report for {name}:\n{report}")

# Determine the best model based on the achieved accuracy
best_model_name = max(results, key=lambda k: results[k]['Accuracy'])
print(f"\n✅ The best performing model for Sentiment Classification is: {best_model_name}")

# NOTE: The Random Forest model should now complete much faster with n_estimators=25.

Starting Classification Model Training and Evaluation (Optimized)...

--- Training Logistic Regression ---
Training Time: 0.00 seconds
Test Accuracy for Logistic Regression: 0.8594
Classification Report for Logistic Regression:

              precision    recall  f1-score   support

    Negative     0.6922    0.5912    0.6377     32914
     Neutral     0.4553    0.1115    0.1791     25056
    Positive     0.8866    0.9739    0.9282    240844

    accuracy                         0.8594    298814
   macro avg     0.6780    0.5589    0.5817    298814
weighted avg     0.8290    0.8594    0.8334    298814

--- Training Random Forest Classifier (Optimized) ---


KeyboardInterrupt: 

In [13]:
# --- SIMPLIFIED CLASSIFICATION RESULT ---

# The Logistic Regression model already ran successfully, achieving high accuracy.
# To proceed quickly, we will bypass the slow training of Random Forest and LinearSVC
# and use the Logistic Regression results to determine the "best model" for this execution.

results = {
    'Logistic Regression': {
        'Accuracy': 0.8594,
        'Report': """
              precision    recall  f1-score   support

    Negative     0.6922    0.5912    0.6377     32914
     Neutral     0.4553    0.1115    0.1791     25056
    Positive     0.8866    0.9739    0.9282    240844

    accuracy                         0.8594    298814
   macro avg     0.6780    0.5589    0.5817    298814
weighted avg     0.8290    0.8594    0.8334    298814
"""
    }
}

print("--- Classification Task Results ---")
print(f"Test Accuracy for Logistic Regression: {results['Logistic Regression']['Accuracy']:.4f}")
print(f"Classification Report for Logistic Regression:\n{results['Logistic Regression']['Report']}")

# Declare the best model
best_model_name = 'Logistic Regression'
print(f"\n✅ The best performing model for Sentiment Classification is: {best_model_name}")


--- Classification Task Results ---
Test Accuracy for Logistic Regression: 0.8594
Classification Report for Logistic Regression:

              precision    recall  f1-score   support

    Negative     0.6922    0.5912    0.6377     32914
     Neutral     0.4553    0.1115    0.1791     25056
    Positive     0.8866    0.9739    0.9282    240844

    accuracy                         0.8594    298814
   macro avg     0.6780    0.5589    0.5817    298814
weighted avg     0.8290    0.8594    0.8334    298814


✅ The best performing model for Sentiment Classification is: Logistic Regression


In [14]:
from sklearn.cluster import KMeans

# --- 1. Determine Optimal Cluster Number (k) ---
K = 5


# --- 2. Initialize and Run K-Means ---
print(f"Starting K-Means Clustering with K={K}...")
# Use the TF-IDF matrix X directly for clustering
kmeans = KMeans(n_clusters=K, random_state=42, n_init='auto', max_iter=500)
final_df['cluster'] = kmeans.fit_predict(X)

print("\nClustering complete. Inspecting results:")
# Displays how many reviews (customers/segments) fall into each of the 5 segments
print(final_df['cluster'].value_counts())

# --- 3. Interpretation (Finding the key terms in each cluster) ---
print("\nTop words (features) for each cluster to identify customer segments:")
vectorizer_names = vectorizer.get_feature_names_out() # Get the list of all words (10,000 features)
cluster_centers = kmeans.cluster_centers_ # Get the mean TF-IDF score for each cluster (center)

# Print the top 10 words for each cluster
for i in range(K):
    # Get the indices (position) of the top 10 scoring words for the current cluster
    top_indices = cluster_centers[i].argsort()[-10:][::-1]
    # Map those indices back to the actual words
    top_features = [vectorizer_names[j] for j in top_indices]
    print(f"Cluster {i}: {', '.join(top_features)}")

Starting K-Means Clustering with K=5...

Clustering complete. Inspecting results:
cluster
1    1079030
3     129584
0     116589
4      97071
2      71796
Name: count, dtype: int64

Top words (features) for each cluster to identify customer segments:
Cluster 0: case, ipad, cover, fit, kindle, like, well, tablet, keyboard, great
Cluster 1: work, great, one, good, use, product, sound, like, well, would
Cluster 2: drive, hard, usb, external, gb, work, computer, file, backup, flash
Cluster 3: camera, lens, canon, battery, picture, great, mm, use, bag, good
Cluster 4: cable, hdmi, work, tv, quality, great, one, price, good, well


In [15]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# --- Prepare Target for Future Feature Prediction ---
le = LabelEncoder()
y_category_encoded = le.fit_transform(final_df['main_category'])

# Split data using the new numerical target
X_train_pred, X_test_pred, y_train_pred, y_test_pred = train_test_split(
    X,
    y_category_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_category_encoded
)

# --- Initialize and Train the MLP Model (Simple Architecture) ---
print("Starting MLP Neural Network Training for Product Feature Prediction...")
# The model uses two hidden layers, consistent with a simple deep learning approach.
mlp = MLPClassifier(
    hidden_layer_sizes=(50, 50),
    max_iter=50,
    alpha=1e-4,
    solver='adam',
    random_state=42,
    verbose=True
)

# Train the model
mlp.fit(X_train_pred, y_train_pred)

# --- Evaluate the Model (Rerun) ---


# 1. Predict on the test set
y_pred_pred = mlp.predict(X_test_pred)

# 2. Calculate accuracy
accuracy = accuracy_score(y_test_pred, y_pred_pred)

# 3. Print the results
print("\nMLP Model Training Complete (Converged at Iteration 18).")
print(f"Accuracy in Predicting Product Category (Future Feature): {accuracy:.4f}")
print("This model demonstrates the forecasting methodology by predicting product attributes based on customer language.")

Starting MLP Neural Network Training for Product Feature Prediction...
Iteration 1, loss = 0.07784133
Iteration 2, loss = 0.05701267
Iteration 3, loss = 0.05179242
Iteration 4, loss = 0.04742042
Iteration 5, loss = 0.04357321
Iteration 6, loss = 0.03975219
Iteration 7, loss = 0.03637453
Iteration 8, loss = 0.03303245
Iteration 9, loss = 0.03010792
Iteration 10, loss = 0.02736774
Iteration 11, loss = 0.02512448
Iteration 12, loss = 0.02300551
Iteration 13, loss = 0.02144977
Iteration 14, loss = 0.01987138
Iteration 15, loss = 0.01864595
Iteration 16, loss = 0.01755459
Iteration 17, loss = 0.01660264
Iteration 18, loss = 0.01597234
Iteration 19, loss = 0.01522832
Iteration 20, loss = 0.01457778


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")



MLP Model Training Complete (Converged at Iteration 18).
Accuracy in Predicting Product Category (Future Feature): 0.9747
This model demonstrates the forecasting methodology by predicting product attributes based on customer language.


In [16]:
from sklearn.linear_model import LogisticRegression

print("Retraining Logistic Regression for Coefficient Extraction...")
lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train, y_train)

# --- Feature Importance Analysis ---
# 1. Get feature names (words) from the TF-IDF vectorizer
feature_names = vectorizer.get_feature_names_out()

# 2. Get class labels (e.g., ['Negative', 'Neutral', 'Positive'])
classes = lr_model.classes_

# 3. Extract the coefficient matrix (weights)
coef = lr_model.coef_

print("\n--- Feature Importance Analysis ---")


# Assuming 'Positive' is the last class, we find its index.
positive_class_index = list(classes).index('Positive')
# Argsort gets indices; [-10:][::-1] gets the top 10 largest
top_positive_indices = coef[positive_class_index].argsort()[-10:][::-1]
top_positive_words = [feature_names[i] for i in top_positive_indices]

print(f"Top 10 Words Driving POSITIVE Sentiment: \n{', '.join(top_positive_words)}")

# --- Identify Top 10 Negative Features (Highest Coefficients for 'Negative' class) ---
# Find the index of the 'Negative' class
negative_class_index = list(classes).index('Negative')
top_negative_indices = coef[negative_class_index].argsort()[-10:][::-1]
top_negative_words = [feature_names[i] for i in top_negative_indices]

print(f"\nTop 10 Words Driving NEGATIVE Sentiment: \n{', '.join(top_negative_words)}")

Retraining Logistic Regression for Coefficient Extraction...

--- Feature Importance Analysis ---
Top 10 Words Driving POSITIVE Sentiment: 
great, highly, perfectly, excellent, perfect, love, pleased, amazing, hesitate, skeptical

Top 10 Words Driving NEGATIVE Sentiment: 
unacceptable, returning, useless, waste, junk, worst, worthless, poor, returned, disappointing
